In [1]:
from urllib.request import urlopen
import timm
import torch
import zipfile,os
from PIL import Image
from pathlib import Path
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

In [2]:
pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 38.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 92.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-sto

In [3]:
pip install scikit-learn==1.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 90.4 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.3.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib==3.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 98.4 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.2
    Uninstalling matplotlib-3.7.2:
      Successfully uninstalled matplotlib-3.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="B5AgQrixtU1h66djOZMm")
project = rf.workspace("digihack").project("blink-classification-a5pan")
version = project.version(1)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Blink-Classification-1 in folder:: 100%|██████████| 2911/2911 [00:00<00:00, 8146.43it/s]


In [6]:
train_dir = dataset.location + "/train"
test_dir = dataset.location + "/test"
val_dir = dataset.location + "/valid"

In [7]:
model = timm.create_model(
    'mobilevitv2_200.cvnets_in1k',
    pretrained=True,
    num_classes=2,
)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
trans = timm.data.create_transform(**data_config, is_training=False)

model.safetensors:   0%|          | 0.00/73.9M [00:00<?, ?B/s]

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label  # Ensure this returns a tuple of (image, label)


In [11]:
transform = transforms.Compose([
    transforms.Resize((480,480)),
    transforms.ToTensor(),           # Convert images to PyTorch tensors
])

# Create an instance of the CustomDataset
dataset = CustomDataset(data_dir=train_dir, transform=transform)

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)
for data in train_loader:
    print(data[0].shape)  # This will show you the structure of the data being returned
    inputs, targets = data  # Unpack only if it has the correct structure
    break

torch.Size([16, 3, 480, 480])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Definisikan device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_val_loss = float('inf') 
save_path = "./best_model.pth"
# Asumsikan model sudah didefinisikan
# model = ...
model.to(device) # Pindahkan model ke device

# Definisikan loss function
criterion = nn.CrossEntropyLoss()
criterion.to(device)  # <-- TAMBAHKAN BARIS INI untuk memindahkan loss function ke GPU

# Definisikan optimizer (setelah model dipindahkan ke device)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
# --- 1. Buat DataLoader untuk Validasi ---
# Letakkan ini di luar loop training, bersama dengan definisi train_loader Anda.
try:
    val_dataset = ImageFolder(root=val_dir, transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
except NameError:
    print("Pastikan variabel 'val_dir' dan 'transform' sudah didefinisikan.")
    # Atur val_loader ke None atau handle errornya agar tidak crash
    val_loader = None


num_epochs = 200 # Contoh jumlah epoch
# Training loop
for epoch in range(num_epochs):
    
    # ====================================================================
    #                            TRAINING LOOP
    # ====================================================================
    model.train()  # Set the model to training mode
    epoch_loss = 0
    correct_predictions = 0
    total_samples = 0
    all_targets = []
    all_preds = []
    
    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training")
    
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total_samples += targets.size(0)
        correct_predictions += (predicted == targets).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
        
        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    # Hitung dan cetak metrik training setelah satu epoch selesai
    avg_loss = epoch_loss / len(train_loader)
    accuracy = correct_predictions / total_samples * 100
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)

    train_metrics = (f"Epoch [{epoch+1}/{num_epochs}], "
                     f"Train Loss: {avg_loss:.4f}, "
                     f"Train Accuracy: {accuracy:.2f}%, "
                     f"Train F1-Score: {f1:.4f}")
    
    print(train_metrics)
    
    # ====================================================================
    #                           VALIDATION LOOP
    # ====================================================================
    if val_loader:
        model.eval()  # PENTING: Ubah model ke mode evaluasi

        val_loss = 0
        val_correct = 0
        val_total = 0
        all_val_targets = []
        all_val_preds = []

        with torch.no_grad():  # PENTING: Nonaktifkan perhitungan gradien
            val_progress_bar = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation")
            for inputs, targets in val_progress_bar:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += targets.size(0)
                val_correct += (predicted == targets).sum().item()
                
                all_val_preds.extend(predicted.cpu().numpy())
                all_val_targets.extend(targets.cpu().numpy())
        avg_val_loss = val_loss / len(val_loader)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss  # Perbarui nilai loss terbaik
            torch.save(model.state_dict(), save_path) # Simpan model state_dict
            print(f"✅ Model terbaik baru disimpan di epoch {epoch+1} dengan Val Loss: {best_val_loss:.4f}")

        # Hitung dan cetak metrik validasi
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = val_correct / val_total * 100
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(all_val_targets, all_val_preds, average='weighted', zero_division=0)

        val_metrics = (f"             " # Spasi untuk meluruskan
                       f"Val Loss: {avg_val_loss:.4f},  "
                       f"Val Accuracy: {val_accuracy:.2f}%,  "
                       f"Val F1-Score: {val_f1:.4f}\n") # Tambah baris baru
        
        print(val_metrics)
torch.save(model.state_dict(), "./last_model.pth") # Simpan model state_dict

Epoch [1/1] Training:  58%|█████▊    | 92/159 [01:23<01:06,  1.01it/s, loss=0.119] 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_dataset = ImageFolder(root=test_dir, transform = transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []


# Perform forward pass
#outputs = model(inputs)

# Get predicted class indices
#_, preds = torch.max(outputs, dim=1)

# Compare predictions with ground truth
#correct_predictions += (preds == targets).sum().item()
#total_samples += targets.size(0)

# Store predictions and labels for metrics
#all_targets.extend(targets.tolist())
#all_preds.extend(preds.tolist())

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Tiny test2.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(report)

In [ ]:
# Load best model weights from ./best_model.pth and prepare for inference
import os
import torch
import timm
import torch.nn as nn
from collections import OrderedDict

# Recreate the exact model architecture used in training
model_name = 'mobilevitv2_200.cvnets_in1k'
num_classes = 2

# Reuse existing device if defined, otherwise detect automatically
device = globals().get('device', torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Build base model (no need to download pretrained weights when loading a checkpoint)
base_model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)

ckpt_path = './best_model.pth'
if not os.path.exists(ckpt_path):
    raise FileNotFoundError(f'Checkpoint not found at {ckpt_path}. Make sure training saved it successfully.')

# Load checkpoint (supports both raw state_dict and dict with key "state_dict")
state = torch.load(ckpt_path, map_location=device)
state_dict = state.get('state_dict', state) if isinstance(state, dict) else state

# Remove potential DataParallel 'module.' prefix if present
if any(k.startswith('module.') for k in state_dict.keys()):
    cleaned_state_dict = OrderedDict((k.replace('module.', '', 1), v) for k, v in state_dict.items())
else:
    cleaned_state_dict = state_dict

# Load weights (strict=False to allow minor non-critical mismatches)
missing_keys, unexpected_keys = base_model.load_state_dict(cleaned_state_dict, strict=False)

# Optionally wrap with DataParallel if multiple GPUs are available
model = base_model
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(base_model)

model = model.to(device).eval()

print(f'Successfully loaded weights from {ckpt_path} onto {device}.')
if missing_keys:
    print('Missing keys:', missing_keys)
if unexpected_keys:
    print('Unexpected keys:', unexpected_keys)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_dataset = ImageFolder(root=test_dir, transform = transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []


# Perform forward pass
#outputs = model(inputs)

# Get predicted class indices
#_, preds = torch.max(outputs, dim=1)

# Compare predictions with ground truth
#correct_predictions += (preds == targets).sum().item()
#total_samples += targets.size(0)

# Store predictions and labels for metrics
#all_targets.extend(targets.tolist())
#all_preds.extend(preds.tolist())

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Tiny test2.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(report)